In [ ]:
import requests
from bs4 import BeautifulSoup
import os
import pandas as pd

In [ ]:
# Indtast stien til den mappe, hvor filerne skal gemmes.
SAVE_PATH = "./"

In [ ]:
def get_soup(html):
    return BeautifulSoup(html, 'html.parser')

In [ ]:
def get_html(url):
    resp = requests.get(url)
    resp.encoding = "utf-8"
    return resp    

In [ ]:
url = "https://kmdvalg.dk/Main/Home/KV"
base_url = url[:url.rfind("/")]
resp = requests.get(url)
resp.encoding = "utf-8"
soup = get_soup(resp.text)

In [ ]:
muncipality_links = soup.find_all(class_="col-sm-12 content-block kmd-list-items")
numb_elements = len(muncipality_links)
if numb_elements != 1:
    print("Warning: a single element could not be found.")
    print(f"Number of elements is: {numb_elements}")
    print("Please check the code!")
muncipality_links = muncipality_links[0]

In [ ]:
resultat_sider = muncipality_links.find_all("a")
if len(resultat_sider) != 98:
    print("Couldn't find exactly 98 muncipalities.")
    print("Check if Greenland is in the election.")


In [ ]:
result = {x.contents[0].strip():os.path.join(x.get("href")) for x in resultat_sider}

In [ ]:
def get_data_for_whole_muncipality(muncipality_name):
    muncipality = result[muncipality_name]
    resp = get_html(muncipality)
    soup = get_soup(resp.text)
    print("Got:", muncipality_name)
    meta = soup.find(class_="table table-responsive table-condensed kmd-table kmd-table-info")
    meta = meta.find_all("tr")
    meta[0]
    get_metadata = lambda list: [x.contents[0] for x in list.find_all("td")]
    meta = [get_metadata(x) for x in meta]
    stemmer_omr = soup.find(class_="col-xs-12 col-sm-6 col-md-8 content-block kmd-parti-list")
    linjer = stemmer_omr.find_all(class_="row table-like-row")
    få_linje = lambda linje_: [x.text for x in linje_.find_all("div")]
    linjer_format = [få_linje(x) for x in linjer[1:]]
    linjer_res = linjer_format[:3]+linjer_format[-1:]

    # Stemmer på afstemningssted

    afst_omr = soup.find(id="vote-areas")
    afst_omr_res = afst_omr.find_all("a")
    afst_omr_res[0].contetn # .__dict__

    afst_omr_dict = {
        x.contents[0]:os.path.join("https://www.kmdvalg.dk/kv/2021/", x["href"]) for x in afst_omr_res
    }
    reses = []
    for område in afst_omr_dict.keys():
        reses.append(
            afstemningssted(afst_omr_dict, område)
        )
    return {
        'meta': meta,
        'kom_niveau': linjer_res, 
        'områder': reses
    }

def afstemningssted(afst_omr_dict, område_):
    omr = afst_omr_dict[område_]
    resp = get_html(omr)
    soup = get_soup(resp.text)
    meta_ = soup.find(class_="table table-responsive table-condensed kmd-table kmd-table-info")
    # print(meta_)
    meta_ = meta_.find_all("tr")
    get_metadata = lambda liste_: [x.contents[0] for x in liste_.find_all("td")]
    meta_ = [get_metadata(x) for x in meta_]

    table = soup.find(class_="col-xs-12 col-sm-6 col-md-8 content-block kmd-parti-list")
    rows = table.find_all(class_="row table-like-row")[1:]
    row = rows[0].find_all("div")
    get_rawdata_row = lambda row: [x.contents for x in row]
    clean_rawdata = lambda row: [
        row[0][0].contents[0],
        row[0][1].contents[0],
        os.path.join(
            base_url, row[0][1]["href"]),
        row[1][0],
        row[2][0],
        row[-1][0].replace(",", ".")
    ]
    comb = lambda row: clean_rawdata(
        get_rawdata_row(row)
    )
    comb(row)
    omr_df = pd.DataFrame(
        data=[
            comb(
                x.find_all("div")
            ) for x in rows
        ],
        columns=[
            'Bogstav',
            'Parti',
            'Url',
            'Stemmetal',
            '(+/-)',
            'pct_udvikling'
        ]
    )
    omr_df.set_index("Bogstav", inplace=True)
    omr_df["område"] = område_
    return [område_, meta_, omr_df]

In [ ]:
def clean_frame(x):
    meta = x[1]
    ant_stemberet = meta[0][1]
    gyldige_stem = meta[-1][1]
    gyldige_stem_udv = meta[-1][2]
    df = x[2]
    df['ant_stemberet'] = ant_stemberet
    df['gyldige_stem'] = gyldige_stem
    df['gyldige_stem_udv'] = gyldige_stem_udv
    return df

def concatter(områder):
    first = True
    for område in områder:
        if first:
            df = clean_frame(område)
            first = False
            continue
        else:
            df = pd.concat([df, clean_frame(område)])
    return df

In [ ]:
for kom in result.keys():
    try:
        res = get_data_for_whole_muncipality(kom)
        df = concatter(res["områder"])
        df['kom'] = kom
        df.to_csv(SAVE_PATH+kom+".csv")
    except IndexError:
        print(kom)